### Dans ce notebook, nous nous sommes interéssés au revenue de chaque client de Google Store.  Nous allons constuire un modèle du machine learning, avec un rmse le plus petit possible,  qui prend comme input l'historique du client et qui prédit comme output son revenu. Pour cela nous allons suivre la méthodologie CRISP-DM illustrée ci-dessous:

![CRISP-DM](https://ashwiniksite.files.wordpress.com/2017/05/datamining-process.png)

### Objectif: Prédire la variable log(1+revenu total d'un client donné)

## Import necessary packages

In [ ]:
import numpy as np 
import pandas as pd 
import os
print(os.listdir("../input"))
import json
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
def load_df(csv_path, nrows=None):
    '''
        csv_path --string-- path to data frame to read 
        returns: a dataFrame where Json columns are read correctly
    '''
    USE_COLUMNS = [
        'channelGrouping', 'date', 'device', 'fullVisitorId', 'geoNetwork',
        'socialEngagementType', 'totals', 'trafficSource', 'visitId',
        'visitNumber', 'visitStartTime', 'customDimensions'
        #'hits'
    ]
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    df = pd.read_csv(csv_path, 
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'}, # Important!!
                     nrows=nrows, usecols=USE_COLUMNS)
    
    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    print(f"Loaded {os.path.basename(csv_path)}. Shape: {df.shape}")
    return df

## Loading the train and test dataFrames

In [ ]:
%%time
train_df = load_df("../input/train_v2.csv")
test_df = load_df("../input/test_v2.csv")

## Display the first five rows 

In [ ]:
train_df.head()

# Data Preparation

## 1) Feature Pre-Selection

In [ ]:
print("these columns are in train dataFrame but not in test dataFrame \n:",train_df.columns.difference(test_df.columns))
print("these columns are in test dataFrame but not in train dataFrame \n:",test_df.columns.difference(train_df.columns))

In [ ]:
#dropping this column
train_df.drop(labels=['trafficSource.campaignCode'],axis=1, inplace = True)

### Dropping columns with unique values

In [ ]:
print('Features with one unique values are :')
print(train_df.columns[train_df.nunique() == 1])
train_df.drop(labels = train_df.columns[train_df.nunique() == 1],axis = 1, inplace = True)
test_df.drop(labels = train_df.columns[train_df.nunique() == 1],axis = 1,inplace = True)

### Removing visitStartTime, isMobile, customDimensions, totals.totalTransactionRevenue

In [ ]:
# Avoid Data Leakage
train_df.drop(labels = ['visitId','visitStartTime'], axis = 1, inplace = True)
test_df.drop(labels = ['visitId','visitStartTime'], axis = 1, inplace = True)

In [ ]:
#device.isMobile is a duplicate variable as the detail is covered in deviceCategory
train_df.drop(labels=['device.isMobile'], axis=1, inplace = True)
test_df.drop(labels=['device.isMobile'], axis=1,inplace = True)

In [ ]:
train_df = train_df.drop(labels=['customDimensions','totals.totalTransactionRevenue'],axis=1)
test_df = test_df.drop(labels=['customDimensions','totals.transactionRevenue','totals.totalTransactionRevenue'],axis=1)

## 2) Missing values

### helper function to find missing values

In [ ]:

def missing_values(data):
    total = data.isnull().sum().sort_values(ascending = False) # getting the sum of null values and ordering
    percent = (data.isnull().sum() / data.isnull().count() * 100 ).sort_values(ascending = False) #getting the percent and order of null
    df = pd.concat([total, percent], axis=1, keys=['Total', 'Percent']) # Concatenating the total and percent
    print("Total columns at least one Values: ")
    print (df[~(df['Total'] == 0)]) 
    
    print("\n Total of Sales % of Total: ", round((train_df[train_df['totals.transactionRevenue'] != np.nan]['totals.transactionRevenue'].count() / len(train_df['totals.transactionRevenue']) * 100),4))
    
    return 

missing_values(train_df)

### Our target variable 'totals.transactionRevenue' presents 98.91% of missing  values, let's explore it

In [ ]:
#converting from object to float
train_df['totals.transactionRevenue'] = train_df['totals.transactionRevenue'].astype('float')

### basic statistics about the target variable : min,mean,median,max

In [ ]:
print("Transaction Revenue Min Value: ", 
      train_df[train_df['totals.transactionRevenue'] > 0]["totals.transactionRevenue"].min()) # printing the min value
print("Transaction Revenue Mean Value: ", 
      train_df[train_df['totals.transactionRevenue'] > 0]["totals.transactionRevenue"].mean()) # mean value
print("Transaction Revenue Median Value: ", 
      train_df[train_df['totals.transactionRevenue'] > 0]["totals.transactionRevenue"].median()) # median value
print("Transaction Revenue Max Value: ", 
      train_df[train_df['totals.transactionRevenue'] > 0]["totals.transactionRevenue"].max()) # the max value

plt.figure(figsize=(14,5))
plt.subplot(1,2,1)

#histogram of log(revenue+1)
ax = sns.distplot(np.log(train_df[train_df['totals.transactionRevenue'] > 0]["totals.transactionRevenue"] + 1), bins=40, kde=True)
ax.set_xlabel('Transaction RevenueLog', fontsize=15) 
ax.set_ylabel('Distribuition', fontsize=15)
ax.set_title("Distribuition of Revenue Log", fontsize=20) 

plt.subplot(1,2,2)
# ordering the total of users and seting the values of transactions to understanding 
plt.scatter(range(train_df.shape[0]), np.sort(train_df['totals.transactionRevenue'].values))
plt.xlabel('Index', fontsize=15) 
plt.ylabel('Revenue value', fontsize=15) 
plt.title("Revenue Value Distribution", fontsize=20) 

plt.show()

### what we can take from here is:
###  - that the rule 80/20 been proven to be true, even worse, 98.9% of clients do not generate any revenue.
###  - the log function has allowed us to have a normal distribution over the revenue with mean value around 18


### helper function to check outliers

In [ ]:
def checkOutliers(data): 

    # calculating mean and std of the array
    data_mean, data_std = np.mean(data), np.std(data)

    # seting the cut line to both higher and lower values
    cut_line = data_std * 3

    #Calculating the higher and lower cut values
    lower, upper = data_mean - cut_line, data_mean + cut_line

    # creating an array of lower, higher and total outlier values 
    outliers_lower = [x for x in data if x < lower]
    outliers_higher = [x for x in data if x > upper]
    outliers_total = [x for x in data if x < lower or x > upper]

    # array without outlier values
    outliers_removed = [x for x in data if x > lower and x < upper]
    
    print('Identified lowest outliers: %d' % len(outliers_lower)) 
    print('Identified upper outliers: %d' % len(outliers_higher)) 
    print('Identified outliers: %d' % len(outliers_total)) 
    print('Non-outlier observations: %d' % len(outliers_removed))
    print("Total percentual of Outliers: ", round((len(outliers_total) / len(outliers_removed) )*100, 4)) 
    
    return

In [ ]:
print("Outliers in totals.transactionRevenue: \n", "\n")
checkOutliers(train_df['totals.transactionRevenue'])

## 3) Visualization

### in this part we are going to visualize some of the variables present in the training data 

### Channel Grouping

### helper function for visualization

In [ ]:
def visualizeMostFrequent(columnName,top = 5):
    print("Percentual of Browser usage: \n ")
    print(train_df[columnName].value_counts()[:top])
    
    fig = plt.figure(figsize = (10,8))
    plt.subplot(2,1,1)
    sns.countplot(train_df[train_df[columnName]\
                       .isin(train_df[columnName]\
                             .value_counts()[:top].index.values)][columnName], palette="hls") 
    plt.title("TOP "+ str(top) +" Most Frequent "+ columnName, fontsize=20) 
    plt.ylabel("Count", fontsize=16) 
    plt.xticks(rotation=45) 
    fig.tight_layout()
    plt.subplot(2,1,2)
    
    #It's another way to plot our data. using a variable that contains the plot parameters
    ax = sns.boxenplot(x=columnName, y='totals.transactionRevenue', 
                       data=train_df[(train_df[columnName].isin((train_df[columnName].value_counts()[:top].index.values))) &
                                      train_df['totals.transactionRevenue'] > 0])
    ax.set_title(columnName+' Name by Transactions Revenue', fontsize=20) # title and fontsize
    ax.set_xticklabels(ax.get_xticklabels(),rotation=45) # It's the way to rotate the xticks when we use variable to our graphs
    ax.set_xlabel(columnName+' Names', fontsize=18)
    ax.set_ylabel('TransactionRevenue(log)', fontsize=18) 
    plt.show() 
    
    return
    

In [ ]:
visualizeMostFrequent('channelGrouping')

### As we can see, Organic search is the most frequent channel grouping. However, this does not mean it is the one that generates the most of  the revenue

### Browsers

In [ ]:
visualizeMostFrequent('device.browser')

# Device Category

In [ ]:
visualizeMostFrequent('device.deviceCategory', top = 3)

# Operating System

In [ ]:
visualizeMostFrequent('device.operatingSystem', top = 5)

In [ ]:
# we fill missing values in totals.transactionRevenue with 0
train_df['totals.transactionRevenue'].fillna(0, inplace = True)

In [ ]:
print('Total records: ', len(train_df), 'customers do not contribute for revenue: ',train_df['totals.transactionRevenue'].isna().sum(), \
      'customers contributing revenue: ',len(train_df) - train_df['totals.transactionRevenue'].isna().sum())

### Exploring the time and number visits

In [ ]:
train_df['date']= pd.to_datetime(train_df['date'],format='%Y%m%d')
test_df['date']= pd.to_datetime(test_df['date'],format='%Y%m%d')

In [ ]:
#Lets analyze how the visits and transactionRevenue trend with date
#Yearly pattern

plot_df = train_df.groupby(by=['date']).agg({'fullVisitorId':'count',
                                        'totals.transactionRevenue':'sum'}).reset_index()

fig = plt.figure(figsize=(15, 8))
plt.subplot(2, 1, 1)
plt.plot(pd.to_datetime(plot_df['date']), plot_df['totals.transactionRevenue'])
plt.ylabel('Revenue')
#plt.xticks(rotation=90)

plt.subplot(2,1,2)
plt.plot(pd.to_datetime(plot_df['date']),plot_df['fullVisitorId'])
plt.ylabel('Visits')

In [ ]:
#Weekly pattern

plot_df = train_df.groupby(by=['date']).agg({'fullVisitorId':'count','totals.transactionRevenue':'sum'})

plot_df = plot_df.resample('W').mean()

plt.figure(figsize=(15,8))
plt.subplot(2,1,1)
plt.plot(plot_df.index,plot_df['totals.transactionRevenue'])
plt.ylabel('Revenue')

plt.subplot(2,1,2)
plt.plot(plot_df.index,plot_df['fullVisitorId'])
plt.ylabel('Visits');

Number of visits drasitically increases from October and reduces in December. But that didnt contribute much on revenue. As only one year of data available, we cannot identify any YoY pattern of data.

In [ ]:
train_df.drop(labels = ['date'], axis = 1, inplace = True)
test_df.drop(labels = ['date'], axis = 1, inplace = True)

### Exploring the geoNetwork column

### computing the occurence of each category present in each of geoNetwork subcolumns

In [ ]:
geofields = ['geoNetwork.city','geoNetwork.continent',
             'geoNetwork.country','geoNetwork.metro',
             'geoNetwork.networkDomain','geoNetwork.region',
             'geoNetwork.subContinent']

for fields in geofields:
    temp_df = train_df.groupby(by=fields).size().sort_values(ascending=False).head(10)
    print(temp_df)
    print('*'*30)

In [ ]:
def getplot(df1, df2, groupfield):
    fig =plt.figure(figsize=(15,8))
    plt.subplot(2,1,1)
    plot_df = df1.groupby(by=[groupfield])['totals.transactionRevenue'].size().reset_index()
    plot_df = plot_df.sort_values(by=['totals.transactionRevenue'], ascending=False).head(10)
    plt1 = plt.bar(plot_df[groupfield],plot_df['totals.transactionRevenue'])

    plt.subplot(2,1,2)
    plot_df = df2.groupby(by=[groupfield])['totals.transactionRevenue'].size().reset_index()
    plot_df = plot_df.sort_values(by=['totals.transactionRevenue'], ascending=False).head(10)
    plt2 = plt.bar(plot_df[groupfield],plot_df['totals.transactionRevenue'])
    for ax in fig.axes:
        plt.sca(ax)
        plt.xticks(rotation=15)
    #fig.tight_layout()
    return plt1, plt2

### creating two seperate Data Frames 

In [ ]:
transaction_df = train_df[train_df['totals.transactionRevenue'] >0.0]
nontransaction_df = train_df[train_df['totals.transactionRevenue'] == 0.0]

In [ ]:
getplot(transaction_df,nontransaction_df,'geoNetwork.country');


- We can say that if country == Venezuela, Taiwan,Mexico,Australia,Signapore then the revenue is not zero
- There is not a clear relation between revenue and country

In [ ]:
getplot(transaction_df,nontransaction_df,'geoNetwork.city');

In [ ]:
getplot(transaction_df,nontransaction_df,'geoNetwork.continent');

In [ ]:
getplot(transaction_df,nontransaction_df,'geoNetwork.metro');

the majority of the metro column is not avaibable in the data set

In [ ]:
getplot(transaction_df,nontransaction_df,'geoNetwork.networkDomain');

not set is dominant over all other categorities

In [ ]:
getplot(transaction_df,nontransaction_df,'geoNetwork.subContinent');

Lets drop labels other than city and subcontinent. As city is the lower granular level in geo. As most values in city is 'not available' lets keep another variable subcontinent as an additional field.

In [ ]:
train_df.drop(labels=['geoNetwork.region','geoNetwork.networkDomain','geoNetwork.metro',
                              'geoNetwork.continent','geoNetwork.country'],
                      axis=1, inplace = True)
test_df.drop(labels=['geoNetwork.region','geoNetwork.networkDomain','geoNetwork.metro',
                                'geoNetwork.continent','geoNetwork.country'],
                        axis=1,inplace = True)

In [ ]:
getplot(transaction_df,nontransaction_df,'trafficSource.source')

In [ ]:
train_df.drop(labels=['trafficSource.adwordsClickInfo.adNetworkType', 'trafficSource.adwordsClickInfo.gclId',
                               'trafficSource.adwordsClickInfo.page', 'trafficSource.adwordsClickInfo.slot',
                               'trafficSource.medium', 'trafficSource.medium', 'trafficSource.referralPath','trafficSource.source','trafficSource.keyword','trafficSource.adContent'],
                       axis=1,inplace = True)
test_df.drop(labels=['trafficSource.adwordsClickInfo.adNetworkType', 'trafficSource.adwordsClickInfo.gclId',
                                 'trafficSource.adwordsClickInfo.page', 'trafficSource.adwordsClickInfo.slot',
                                 'trafficSource.medium', 'trafficSource.medium', 'trafficSource.referralPath','trafficSource.source','trafficSource.keyword','trafficSource.adContent'],
                         axis=1,inplace = True)
print(train_df.shape)
train_df.head()

In [ ]:
train_df.fillna('0',inplace=True)
test_df.fillna('0',inplace=True)

## 4) Encoding Part 

### helper function to convert to dummies variables

In [ ]:
def convert_category_todummies(df,field):
    #print('Processing ', field)
    dummy_df = pd.get_dummies(df[field])
    df = pd.concat([df,dummy_df],axis=1)
    df.drop(labels=[field],axis=1,inplace=True)
    return df

In [ ]:
def convert_category_tolevel(df,field):
    df[field],index = pd.factorize(df[field])
    return df

In [ ]:
toDrop = list(test_df.columns.difference(train_df.columns))
test_df.drop(labels = toDrop, axis = 1, inplace = True)

In [ ]:
toConvert = ['totals.pageviews','totals.hits','totals.sessionQualityDim','totals.timeOnSite','totals.transactions']

for column in toConvert:
    train_df[column] = train_df[column].astype(int)
    test_df[column] = test_df[column].astype(int)

### merging the two data Frames

In [ ]:
train_size = train_df.shape[0]
merged_df = pd.concat([train_df,test_df])

print('Before: ', merged_df.shape)
columns = merged_df.columns
for fields in columns:
    if merged_df[fields].dtype == 'object' and fields not in ['fullVisitorId','method']:
        print('Unique values for ', fields, len(merged_df[fields].unique()), merged_df[fields].unique())
        if len(merged_df[fields].unique()) > 40:
            print('Level conversion')
            merged_df[fields] = convert_category_tolevel(merged_df,fields)
            merged_df[fields] = merged_df[fields].astype('int')
        else:
            print('One hot conversion')
            merged_df = convert_category_todummies(merged_df,fields)
            #merged_df = merged_df.drop(labels=[fields],axis=1)
print('After: ', merged_df.shape)

In [ ]:
merged_df['totals.transactionRevenue'] = np.log1p(merged_df['totals.transactionRevenue'])
train_df = merged_df.iloc[:train_size]
test_df = merged_df[train_size:]
test_df.drop(labels=['totals.transactionRevenue'],axis=1, inplace = True)
print(train_df.shape, test_df.shape)

In [ ]:
#we need to predict log revenue per customer. Lets group by full visitor id
train_df = train_df.groupby(by=['fullVisitorId']).mean()
train_df.shape

### creating the target variable 

In [ ]:
train_label = train_df['totals.transactionRevenue']
train_df.drop(labels=['totals.transactionRevenue'],axis=1, inplace = True)

### normalizing the train and test data 

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler1 = MinMaxScaler()
scaled_df = scaler1.fit_transform(train_df.values)
train_df = pd.DataFrame(scaled_df, columns=train_df.columns)

In [ ]:
test_df = test_df.groupby(by=['fullVisitorId']).mean()
visitorid = test_df.index
scaled_test = scaler1.transform(test_df)
test_df = pd.DataFrame(scaled_test,columns=test_df.columns)

# Modeling

## 1) Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression(n_jobs = -1)
lr.fit(train_df,train_label)

### helper function to compute the error

In [ ]:
import numpy as np 
from sklearn.metrics import mean_squared_error
def compute_rmse(y_pred,y_truth):
    return np.sqrt(mean_squared_error(y_pred,y_truth))

In [ ]:
print("LR--RMSE-- on TRAIN: ",compute_rmse(train_label,lr.predict(train_df)))

## 2) Decision Tree Regression

In [ ]:
from sklearn.tree import DecisionTreeRegressor 
from sklearn.model_selection import cross_val_score
dtr = DecisionTreeRegressor(random_state = 2018)
dtr.fit(train_df,train_label)

In [ ]:
print("DecisionTreeRegressor--RMSE-- on TRAIN: ",compute_rmse(train_label,dtr.predict(train_df)))

### helper function to make submission

In [ ]:
def make_submission(y_pred_test,file_name):
    submission = pd.DataFrame({'fullVisitorId':visitorid,'PredictedLogRevenue':y_pred_test})
    submission['fullVisitorId']= submission['fullVisitorId'].astype(str)
    submission['PredictedLogRevenue']=submission['PredictedLogRevenue'].apply(lambda x: 0 if x<0 else x)

    submission.to_csv(file_name+'.csv',index=False)
    
    return 

In [ ]:
make_submission(dtr.predict(test_df),'decisionTree')

### We encountered overfitting problem 

## 3) Random Forests

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rfr = RandomForestRegressor(n_estimators = 20,random_state = 2018,n_jobs = -1)
rfr.fit(train_df,train_label)

In [ ]:
print("RandomForestRegressor--RMSE-- on TRAIN: ",compute_rmse(train_label,rfr.predict(train_df)))

In [ ]:
make_submission(rfr.predict(test_df),'randomForest')

### still overfitting

In [ ]:
import lightgbm as lgb
lgb_data = lgb.Dataset(train_df, label=train_label)
lgb_params = {
        "objective" : "regression",
        "metric" : "rmse",
        "num_leaves" : 40,
        "max_depth" : 10,
        "boosting" : "gbdt",
        "learning_rate" : 0.2,
        "bagging_fraction" : 0.7,
        "feature_fraction" : 0.95,
        'reg_alpha':1,
        "bagging_seed" : 42, "seed":42}
num_boost_rounds_lgb = 1000
model_lgb = lgb.train(lgb_params, lgb_data,num_boost_round=num_boost_rounds_lgb)

In [ ]:
print("GradientBoostingRegressor--RMSE-- on TRAIN: ",compute_rmse(train_label,model_lgb.predict(train_df)))

In [ ]:
make_submission(model_lgb.predict(test_df),'lgb')

### better performance on the test set 

### Is stacking possible, let's compute correlations 

In [ ]:
y1 = lr.predict(train_df)
y2 = dtr.predict(train_df)
y3 = rfr.predict(train_df)
y4 = model_lgb.predict(train_df)

In [ ]:
y1 = pd.DataFrame(y1)
y2 = pd.DataFrame(y2)
y3 = pd.DataFrame(y3)
y4 = pd.DataFrame(y4)

In [ ]:
preds = pd.concat([y1,y2,y3,y4,], axis = 1)
preds.columns = ['linearRegression','decisionTree','randomForest','gradientBoosting']

In [ ]:
corrs = preds.corr()
corrs.style

### all the outputs are highly correlated

In [ ]:
from sklearn.model_selection import GridSearchCV

gridParams = {
    'learning_rate': [0.2,0.25,0.3],
    'n_estimators': [1000,1100],
    'num_leaves': [30,40],
    'boosting_type' : ['gbdt'],
    'objective' : ['regression'],
    'feature_fraction' : [0.95],
    'bagging_seed':[42],'seed':[42]
    }
mdl = lgb.LGBMRegressor(boosting_type= 'gbdt',objective = "regression",metric = "rmse",
        num_leaves = 40,
        max_depth = 100,
        boosting = "gbdt",
        learning_rate = 0.2,
        bagging_fraction = 0.7,
        feature_fraction = 0.95,
        bagging_seed = 42, seed = 42)

grid = GridSearchCV(mdl, gridParams,verbose=0,cv=3,n_jobs= -1)